In [1]:
!pip install pymongo gridfs

ERROR: Could not find a version that satisfies the requirement gridfs (from versions: none)

[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip
ERROR: No matching distribution found for gridfs


In [6]:
import os
from pymongo import MongoClient
import gridfs

# Connect to MongoDB
client = MongoClient("mongodb://localhost:27017/")  
db = client["fashion_db"]
fs = gridfs.GridFS(db)

# Function to Upload Image from File Path
def upload_image(image_path):
    with open(image_path, "rb") as img_file:
        file_id = fs.put(img_file, filename=os.path.basename(image_path), content_type="image/jpeg")
    return file_id

# Folder Containing Subfolders
main_folder = r"C:\Users\Samridhaa\OneDrive\Desktop\BD\Train"

# Iterate Over All Folders and Images
for subfolder in os.listdir(main_folder):
    subfolder_path = os.path.join(main_folder, subfolder)
    
    if os.path.isdir(subfolder_path):  # Check if it's a folder
        for image_file in os.listdir(subfolder_path):
            image_path = os.path.join(subfolder_path, image_file)

            if image_path.lower().endswith((".jpg", ".jpeg", ".png")):  # Check for image files
                file_id = upload_image(image_path)
                print(f"Uploaded {image_file} -> ID: {file_id}")


Uploaded 041c6bde-e737-46fd-9586-984c1503941f.jpg -> ID: 67ed368ca71a0bb74dc38609
Uploaded 053689bb-5928-489b-aeaf-dfb969b05e93.jpg -> ID: 67ed368ca71a0bb74dc3860b
Uploaded 0546d34a-a369-4723-8d3a-74c0f51af26c.jpg -> ID: 67ed368ca71a0bb74dc3860d
Uploaded 0b5f7f85-fa44-4406-91ed-6b730e4b6fa1.jpg -> ID: 67ed368ca71a0bb74dc3860f
Uploaded 14c8710731d28a4c3505103d31ec2d4c.jpg -> ID: 67ed368ca71a0bb74dc38611
Uploaded 1c22dfec-9633-4d10-88f5-b6cad5b27dfc.jpg -> ID: 67ed368ca71a0bb74dc38613
Uploaded 1da10477-cb53-4506-b54c-1ad949002e35.jpg -> ID: 67ed368ca71a0bb74dc38615
Uploaded 1_31.jpg -> ID: 67ed368ca71a0bb74dc38617
Uploaded 20145871-01.jpg -> ID: 67ed368ca71a0bb74dc38619
Uploaded 22cc2840-9a64-4d9f-8dbe-00e81fa4b2c1.jpg -> ID: 67ed368ca71a0bb74dc3861b
Uploaded 2456b899-8146-4823-951a-efe3ab0c2a5b.jpg -> ID: 67ed368ca71a0bb74dc3861d
Uploaded 32c91193-3fe7-48a1-8867-01c55e4f3985.jpg -> ID: 67ed368ca71a0bb74dc3861f
Uploaded 343041d5f0920e985148c7e66baf1b1a.jpg -> ID: 67ed368ca71a0bb74dc38621

In [8]:
!pip install torch torchvision pinecone pymongo gridfs

ERROR: Could not find a version that satisfies the requirement gridfs (from versions: none)

[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip
ERROR: No matching distribution found for gridfs


In [1]:
import pinecone
print(pinecone.__version__)  # Check installed version


6.0.2


In [19]:
import os
import torch
import torchvision.transforms as transforms
import clip
from PIL import Image
from pymongo import MongoClient
import gridfs
import io
from pinecone import Pinecone  
from pinecone import ServerlessSpec

# Initialize CLIP Model
device = "cuda" if torch.cuda.is_available() else "cpu"
model, preprocess = clip.load("ViT-B/32", device=device)

# Connect to MongoDB
client = MongoClient("mongodb://127.0.0.1:27017/")
db = client["fashion_db"]
fs = gridfs.GridFS(db)

# Initialize Pinecone v3+
pc = Pinecone(api_key="pcsk_4zJ7TF_AD6fdpSsj1eMg6TiJthakzEKXg6wkfw4u4j3B5T67bqBZkBd1CFY3dVxSbHnUrg")  

# Check if index exists, otherwise create it
if "fashion-images" not in [i.name for i in pc.list_indexes()]:
    
    pc.create_index(
    name="fashion-images",
    dimension=512,  # CLIP feature vector size
    metric="cosine",
    spec=ServerlessSpec(cloud="aws", region="us-east-1")  # Use the free-tier region
    )

    print("Index created successfully!")

index = pc.Index("fashion-images")  

# Function to Extract Features
def extract_features(image_data):
    image = Image.open(io.BytesIO(image_data)).convert("RGB")
    image = preprocess(image).unsqueeze(0).to(device)

    with torch.no_grad():
        features = model.encode_image(image)
        features /= features.norm(dim=-1, keepdim=True)  # Normalize

    return features.cpu().numpy().flatten()

# Function to Process Images from MongoDB
def process_and_store_images():
    for grid_out in fs.find():  # Iterate through all stored images
        image_data = grid_out.read()
        image_id = str(grid_out._id)

        # Extract Features
        features = extract_features(image_data)

        # Store Features in Pinecone
        index.upsert(vectors=[(image_id, features.tolist(), {"image_id": image_id})])
        print(f"Stored {grid_out.filename} in Pinecone with ID: {image_id}")

# Run the function
process_and_store_images()


Index created successfully!
Stored 041c6bde-e737-46fd-9586-984c1503941f.jpg in Pinecone with ID: 67ed368ca71a0bb74dc38609
Stored 053689bb-5928-489b-aeaf-dfb969b05e93.jpg in Pinecone with ID: 67ed368ca71a0bb74dc3860b
Stored 0546d34a-a369-4723-8d3a-74c0f51af26c.jpg in Pinecone with ID: 67ed368ca71a0bb74dc3860d
Stored 0b5f7f85-fa44-4406-91ed-6b730e4b6fa1.jpg in Pinecone with ID: 67ed368ca71a0bb74dc3860f
Stored 14c8710731d28a4c3505103d31ec2d4c.jpg in Pinecone with ID: 67ed368ca71a0bb74dc38611
Stored 1c22dfec-9633-4d10-88f5-b6cad5b27dfc.jpg in Pinecone with ID: 67ed368ca71a0bb74dc38613
Stored 1da10477-cb53-4506-b54c-1ad949002e35.jpg in Pinecone with ID: 67ed368ca71a0bb74dc38615
Stored 1_31.jpg in Pinecone with ID: 67ed368ca71a0bb74dc38617
Stored 20145871-01.jpg in Pinecone with ID: 67ed368ca71a0bb74dc38619
Stored 22cc2840-9a64-4d9f-8dbe-00e81fa4b2c1.jpg in Pinecone with ID: 67ed368ca71a0bb74dc3861b
Stored 2456b899-8146-4823-951a-efe3ab0c2a5b.jpg in Pinecone with ID: 67ed368ca71a0bb74dc3861d

In [21]:
import torch
import torchvision.transforms as transforms
import clip
from PIL import Image
import io
from pinecone import Pinecone

# Initialize CLIP Model
device = "cuda" if torch.cuda.is_available() else "cpu"
model, preprocess = clip.load("ViT-B/32", device=device)

# Initialize Pinecone client
pc = Pinecone(api_key="pcsk_4zJ7TF_AD6fdpSsj1eMg6TiJthakzEKXg6wkfw4u4j3B5T67bqBZkBd1CFY3dVxSbHnUrg")
index = pc.Index("fashion-images")

# Function to extract features from an image
def extract_features(image_data):
    image = Image.open(io.BytesIO(image_data)).convert("RGB")
    image = preprocess(image).unsqueeze(0).to(device)

    with torch.no_grad():
        features = model.encode_image(image)
        features /= features.norm(dim=-1, keepdim=True)  # Normalize

    return features.cpu().numpy().flatten()

# Function to find similar images
def find_similar_images(image_path, top_k=10):
    with open(image_path, "rb") as f:
        image_data = f.read()

    query_vector = extract_features(image_data)

    # Perform vector search in Pinecone
    results = index.query(vector=query_vector.tolist(), top_k=top_k, include_metadata=True)

    # Print the results
    print("Top 10 similar images:")
    for match in results["matches"]:
        print(f"Image ID: {match['id']}, Score: {match['score']}")

# Example usage
find_similar_images(r"C:\Users\Samridhaa\OneDrive\Desktop\BD\Test\Pants\Image_train_00531.jpg")


Top 10 similar images:
Image ID: 67ed368ca71a0bb74dc38760, Score: 0.86675632
Image ID: 67ed368ea71a0bb74dc38a47, Score: 0.85571903
Image ID: 67ed368da71a0bb74dc387ce, Score: 0.854414761
Image ID: 67ed368da71a0bb74dc3878e, Score: 0.849781
Image ID: 67ed368da71a0bb74dc387d2, Score: 0.845799327
Image ID: 67ed368da71a0bb74dc38774, Score: 0.84054482
Image ID: 67ed368da71a0bb74dc387a0, Score: 0.837491453
Image ID: 67ed368da71a0bb74dc38792, Score: 0.83699
Image ID: 67ed368da71a0bb74dc387ae, Score: 0.836674392
Image ID: 67ed368da71a0bb74dc387c0, Score: 0.83344543
